# PROJETO: Modelagem de dados da SSP-SP / Modeling of SSP-SP data

## João Leopoldo e Silva - fev/2020

#### dataset: https://www.kaggle.com/inquisitivecrow/crime-data-in-brazil
## 2010

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
import joblib

In [6]:
pd.options.display.max_columns = 200

In [7]:
data_2010_1 = pd.read_csv('crime-data-in-brazil/BO_2010_1.csv', low_memory=False)
data_2010_2 = pd.read_csv('crime-data-in-brazil/BO_2010_2.csv', low_memory=False)
data_2010 = pd.concat([data_2010_1, data_2010_2], ignore_index=True)

C:\Users\JOAO\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
rub_dict = joblib.load('rub_dict')

In [8]:
def hora(x):
    try:
        if int(x[:2]) < 6:
            return 'Madrugada'
        elif int(x[:2]) < 12:
            return 'Manhã'
        elif int(x[:2]) < 18:
            return 'Tarde'
        else:
            return 'Noite'
    except:
        return 'Indeterminado'

In [10]:
data_2010['hora'] = data_2010.HORA_OCORRENCIA_BO.apply(hora)

In [11]:
data_2010.sample()

,ANO,ANO_BO,CIDADE,CONDUTA,CONT_PESSOA,COR,DATA_OCORRENCIA_BO,DELEGACIA,DESCR_GRAU_INSTRUCAO,DESCR_PROFISSAO,DESCR_TIPO_PESSOA,DESDOBRAMENTO,FLAG_STATUS,FLAG_STATUS.1,HORA_OCORRENCIA_BO,IDADE_PESSOA,ID_DELEGACIA,LATITUDE,LOGRADOURO,LONGITUDE,MES,NOME_DELEGACIA_CIRC,NOME_DEPARTAMENTO,NOME_DEPARTAMENTO_CIRC,NOME_SECCIONAL,NOME_SECCIONAL_CIRC,NUMERO_LOGRADOURO,NUM_BO,RUBRICA,SEXO_PESSOA,Unnamed: 30,hora
495776,2010,2010,COTIA,NaN,2.0,NaN,30/04/2010,DEL.POL.COTIA,NaN,NaN,Partes,NaN,C,C,19:00,NaN,30403,NaN,RODOVIA RAPOSO TAVARES,NaN,4,02º D.P. COTIA,DEMACRO,DEMACRO,DEL.SEC.CARAPICUIBA,DEL.SEC.CARAPICUIBA,0,3146,Lesão corporal culposa na direção de veículo a...,M,NaN,Noite


In [12]:
target = data_2010.RUBRICA.apply(lambda x: rub_dict[x])

In [14]:
data_2010f = data_2010[['NOME_DEPARTAMENTO_CIRC','NOME_SECCIONAL_CIRC',
                       'IDADE_PESSOA','FLAG_STATUS', 'DESCR_TIPO_PESSOA',
                       'SEXO_PESSOA','COR', 'hora','CONDUTA','DESCR_PROFISSAO','DESCR_GRAU_INSTRUCAO']]

In [15]:
data_2010f = pd.get_dummies(data=data_2010f, columns=['NOME_DEPARTAMENTO_CIRC','NOME_SECCIONAL_CIRC',
                                                       'FLAG_STATUS', 'DESCR_TIPO_PESSOA', 
                                                       'SEXO_PESSOA','COR', 'hora'])

In [17]:
data_2010f.loc[data_2010f['IDADE_PESSOA'] == 'IDADE 15 A 17 ANOS'] = '15'
data_2010f.loc[data_2010f['IDADE_PESSOA'] == 'APARENTAVA MENOR DE IDADE       '] = '0'
data_2010f.loc[data_2010f['IDADE_PESSOA'] == 'IDADE APROX.15 A 16A'] = '15'
data_2010f.loc[data_2010f['IDADE_PESSOA'] == 'IDADE APROX.26 ANOS'] = '26'
data_2010f.loc[data_2010f['IDADE_PESSOA'] == 'IDADE 18A       '] = '18'
data_2010f.loc[data_2010f['IDADE_PESSOA'] == 'IDADE 25 ANOS'] = '25'
data_2010f.loc[data_2010f['IDADE_PESSOA'] == 'IDADE 35 A'] = '35'
data_2010f.loc[data_2010f['IDADE_PESSOA'] == ' TER 25 ANOS                 '] = '25'
data_2010f.loc[data_2010f['IDADE_PESSOA'] == ' 19 ANOS                          '] = '19'
data_2010f.loc[data_2010f['IDADE_PESSOA'] == ' 24 ANOS                     '] = '24'
data_2010f.loc[data_2010f['IDADE_PESSOA'] == '30 ANOS         '] = '30'
data_2010f.loc[data_2010f['IDADE_PESSOA'] == 'IDADE APROX.30A'] = '30'
data_2010f.loc[data_2010f['IDADE_PESSOA'] == 'IDADE APROX.20 ANOS'] = '20'

In [43]:
data_2010f['IDADE_PESSOA'] = pd.to_numeric(data_2010f['IDADE_PESSOA'])

In [44]:
conduta = joblib.load('conduta7')
conduta_2010 = pd.get_dummies(data_2010f.CONDUTA.where(data_2010f.CONDUTA.isin(conduta), other='Other'))

profissao = joblib.load('profissao7')
profissao_2010 = pd.get_dummies(data_2010f.DESCR_PROFISSAO.where(data_2010f.DESCR_PROFISSAO.isin(profissao), other='Other'))

instrucao = joblib.load('instrucao7')
instucao_2010 = pd.get_dummies(data_2010f.DESCR_GRAU_INSTRUCAO.where(data_2010f.DESCR_GRAU_INSTRUCAO.isin(instrucao), other='Other'))

In [45]:
# conduta = data_2008.CONDUTA.value_counts().nlargest(7).index
# joblib.dump(conduta, 'conduta08')

In [46]:
data_2010f = pd.concat([data_2010f, conduta_2010, profissao_2010, instucao_2010], axis=1).drop(columns=['CONDUTA','DESCR_PROFISSAO','DESCR_GRAU_INSTRUCAO'])

In [50]:
data_2010f.sample()

,IDADE_PESSOA,NOME_DEPARTAMENTO_CIRC_DECAP,NOME_DEPARTAMENTO_CIRC_DEMACRO,NOME_SECCIONAL_CIRC_DEL.SEC.1º CENTRO,NOME_SECCIONAL_CIRC_DEL.SEC.2º SUL,NOME_SECCIONAL_CIRC_DEL.SEC.3º OESTE,NOME_SECCIONAL_CIRC_DEL.SEC.4º NORTE,NOME_SECCIONAL_CIRC_DEL.SEC.5º LESTE,NOME_SECCIONAL_CIRC_DEL.SEC.6º SANTO AMARO,NOME_SECCIONAL_CIRC_DEL.SEC.7º ITAQUERA,NOME_SECCIONAL_CIRC_DEL.SEC.8º SAO MATEUS,NOME_SECCIONAL_CIRC_DEL.SEC.CARAPICUIBA,NOME_SECCIONAL_CIRC_DEL.SEC.DIADEMA,NOME_SECCIONAL_CIRC_DEL.SEC.FRANCO DA ROCHA,NOME_SECCIONAL_CIRC_DEL.SEC.GUARULHOS,NOME_SECCIONAL_CIRC_DEL.SEC.MOGI DAS CRUZES,NOME_SECCIONAL_CIRC_DEL.SEC.OSASCO,NOME_SECCIONAL_CIRC_DEL.SEC.S.BERNARDO DO CAMPO,NOME_SECCIONAL_CIRC_DEL.SEC.SANTO ANDRÉ,NOME_SECCIONAL_CIRC_DEL.SEC.TABOÃO DA SERRA,FLAG_STATUS_C,FLAG_STATUS_T,DESCR_TIPO_PESSOA_Adolescente,DESCR_TIPO_PESSOA_Adolescente infrator,DESCR_TIPO_PESSOA_Advogado,DESCR_TIPO_PESSOA_Autor,DESCR_TIPO_PESSOA_Autor/Vitima,DESCR_TIPO_PESSOA_Averiguado,DESCR_TIPO_PESSOA_Condutor,DESCR_TIPO_PESSOA_Criança,DESCR_TIPO_PESSOA_Curador,DESCR_TIPO_PESSOA_Declarante,DESCR_TIPO_PESSOA_Desaparecido,DESCR_TIPO_PESSOA_Indiciado,DESCR_TIPO_PESSOA_Partes,DESCR_TIPO_PESSOA_Representante,DESCR_TIPO_PESSOA_Testemunha,DESCR_TIPO_PESSOA_Vítima,SEXO_PESSOA_,SEXO_PESSOA_F,SEXO_PESSOA_I,SEXO_PESSOA_M,COR_Amarela,COR_BRANCO,COR_Branca,COR_COR NEGRA,COR_MORENO CLARO,COR_MORENO ESCURO,COR_MULATO,COR_NEGRO,COR_Outros,COR_PARDO,COR_Parda,COR_Preta,COR_Vermelha,hora_Indeterminado,hora_Madrugada,hora_Manhã,hora_Noite,hora_Tarde,ESTABELECIMENTO COMERCIAL,INTERIOR DE VEICULO,INTERIOR TRANSPORTE COLETIVO,OUTROS,Other,RESIDENCIA,TRANSEUNTE,VEICULO,AJUDANTE,APOSENTADO(A),AUTONOMO(A),AUXILIAR ADMINISTRATIVO,COMERCIANTE,DESOCUPADO(A),DO LAR,DOMESTICA,ESTUDANTE,GERENTE,MOTO-BOY,MOTORISTA,Other,POLICIAL MILITAR,PROFESSOR(A),VENDEDOR(A),1 Grau completo,1 Grau incompleto,2 Grau completo,2 Grau incompleto,Analfabeto,Other,Superior completo,Superior incompleto
1333350,63.0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0


In [51]:
data_2010f = data_2010f.fillna(0)

In [52]:
data_2010f = data_2010f.apply(pd.to_numeric, downcast='integer')

In [53]:
data_2010f.sample()

,IDADE_PESSOA,NOME_DEPARTAMENTO_CIRC_DECAP,NOME_DEPARTAMENTO_CIRC_DEMACRO,NOME_SECCIONAL_CIRC_DEL.SEC.1º CENTRO,NOME_SECCIONAL_CIRC_DEL.SEC.2º SUL,NOME_SECCIONAL_CIRC_DEL.SEC.3º OESTE,NOME_SECCIONAL_CIRC_DEL.SEC.4º NORTE,NOME_SECCIONAL_CIRC_DEL.SEC.5º LESTE,NOME_SECCIONAL_CIRC_DEL.SEC.6º SANTO AMARO,NOME_SECCIONAL_CIRC_DEL.SEC.7º ITAQUERA,NOME_SECCIONAL_CIRC_DEL.SEC.8º SAO MATEUS,NOME_SECCIONAL_CIRC_DEL.SEC.CARAPICUIBA,NOME_SECCIONAL_CIRC_DEL.SEC.DIADEMA,NOME_SECCIONAL_CIRC_DEL.SEC.FRANCO DA ROCHA,NOME_SECCIONAL_CIRC_DEL.SEC.GUARULHOS,NOME_SECCIONAL_CIRC_DEL.SEC.MOGI DAS CRUZES,NOME_SECCIONAL_CIRC_DEL.SEC.OSASCO,NOME_SECCIONAL_CIRC_DEL.SEC.S.BERNARDO DO CAMPO,NOME_SECCIONAL_CIRC_DEL.SEC.SANTO ANDRÉ,NOME_SECCIONAL_CIRC_DEL.SEC.TABOÃO DA SERRA,FLAG_STATUS_C,FLAG_STATUS_T,DESCR_TIPO_PESSOA_Adolescente,DESCR_TIPO_PESSOA_Adolescente infrator,DESCR_TIPO_PESSOA_Advogado,DESCR_TIPO_PESSOA_Autor,DESCR_TIPO_PESSOA_Autor/Vitima,DESCR_TIPO_PESSOA_Averiguado,DESCR_TIPO_PESSOA_Condutor,DESCR_TIPO_PESSOA_Criança,DESCR_TIPO_PESSOA_Curador,DESCR_TIPO_PESSOA_Declarante,DESCR_TIPO_PESSOA_Desaparecido,DESCR_TIPO_PESSOA_Indiciado,DESCR_TIPO_PESSOA_Partes,DESCR_TIPO_PESSOA_Representante,DESCR_TIPO_PESSOA_Testemunha,DESCR_TIPO_PESSOA_Vítima,SEXO_PESSOA_,SEXO_PESSOA_F,SEXO_PESSOA_I,SEXO_PESSOA_M,COR_Amarela,COR_BRANCO,COR_Branca,COR_COR NEGRA,COR_MORENO CLARO,COR_MORENO ESCURO,COR_MULATO,COR_NEGRO,COR_Outros,COR_PARDO,COR_Parda,COR_Preta,COR_Vermelha,hora_Indeterminado,hora_Madrugada,hora_Manhã,hora_Noite,hora_Tarde,ESTABELECIMENTO COMERCIAL,INTERIOR DE VEICULO,INTERIOR TRANSPORTE COLETIVO,OUTROS,Other,RESIDENCIA,TRANSEUNTE,VEICULO,AJUDANTE,APOSENTADO(A),AUTONOMO(A),AUXILIAR ADMINISTRATIVO,COMERCIANTE,DESOCUPADO(A),DO LAR,DOMESTICA,ESTUDANTE,GERENTE,MOTO-BOY,MOTORISTA,Other,POLICIAL MILITAR,PROFESSOR(A),VENDEDOR(A),1 Grau completo,1 Grau incompleto,2 Grau completo,2 Grau incompleto,Analfabeto,Other,Superior completo,Superior incompleto
165183,34,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0


In [54]:
data_2010f.shape

(1527273, 92)

In [63]:
data_2010f = data_2010f.drop(columns=['COR_BRANCO', 'COR_COR NEGRA','COR_MORENO CLARO','COR_MORENO ESCURO',
                                      'COR_MULATO','COR_NEGRO','COR_PARDO'])

In [64]:
data_2010f.shape

(1527273, 85)

In [65]:
# col_dict = {x: col for x, col in enumerate(data_2010f.columns)}
# col_dict

#### importando e rodando modelo 2007

In [66]:
import lightgbm
from sklearn.model_selection import train_test_split
from yellowbrick.classifier import ROCAUC

In [67]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

In [68]:
X = data_2010f
X.columns = range(len(X.columns))

In [69]:
model = joblib.load('modelo_bo_2007.pkl')
model

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [73]:
#data_2010

In [71]:
data_final_2010 = data_2010[['NUM_BO','ANO_BO','DELEGACIA', 'RUBRICA']]
data_final_2010['proba_leve'] = model.predict_proba(X)[:,0]
data_final_2010['proba_grave'] = model.predict_proba(X)[:,1]

data_final_2010

C:\Users\JOAO\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\JOAO\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,NUM_BO,ANO_BO,DELEGACIA,RUBRICA,proba_leve,proba_grave
0,1,2010,02º D.P. BOM RETIRO,"Furto qualificado (art. 155, §4o.)",0.378652,0.621348
1,1,2010,16º D.P. VILA CLEMENTINO,Lesão corporal culposa na direção de veículo a...,0.003597,0.996403
2,1,2010,16º D.P. VILA CLEMENTINO,Lesão corporal culposa na direção de veículo a...,0.416883,0.583117
3,1,2010,29º D.P. VILA PRUDENTE,"Furto qualificado (art. 155, §4o.)",0.931464,0.068536
4,1,2010,52º D.P. PARQUE S.JORGE,Furto (art. 155),0.210811,0.789189
...,...,...,...,...,...,...
1527268,1273457,2014,DELEGACIA ELETRONICA,Furto (art. 155),0.358353,0.641647
1527269,1288998,2014,DELEGACIA ELETRONICA,Furto (art. 155),0.954315,0.045685
1527270,1334694,2015,DELEGACIA ELETRONICA,Furto (art. 155),0.920833,0.079167
1527271,1335347,2014,DELEGACIA ELETRONICA,Furto (art. 155),0.936277,0.063723


In [74]:
# y_true = transformando tabela rubrica em 0 ou 1

#y_true = data_final_2008.loc[:,14].apply(lambda x: rub_dict[x])

y_true = data_final_2010.RUBRICA.apply(lambda x: rub_dict[x])

In [75]:
y_pred = model.predict(X)
y_pred

array([1, 1, 1, ..., 0, 0, 0], dtype=int64)

In [76]:
pd.DataFrame(confusion_matrix(y_true, y_pred))

,0,1
0,316894,142236
1,77997,990146


In [77]:
from sklearn.metrics import recall_score, accuracy_score

In [79]:
accuracy_score(y_true, y_pred)

0.8557998471785987

In [78]:
recall_score(y_true, y_pred)

# capturei 93% dos casos letais

0.9269788782962581